# Predicting the price of a car

1. Loading and preprocessing

In [16]:
import warnings 
warnings.simplefilter(action='ignore',category = FutureWarning)

In [18]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV 

In [19]:
dataset = "https://raw.githubusercontent.com/ajaythomas200/Machine_Learning_Module_End_Project_Predicting_Car_Price/refs/heads/main/CarPrice_Dataset.csv"

In [20]:
df = pd.read_csv(dataset)
df

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [21]:
# Extract brand from CarName
df['CarBrand'] = df['CarName'].apply(lambda x: x.split()[0])
df.drop(columns=['CarName', 'car_ID'], inplace=True)  #(Eyeball check)-Dropping the unnecessry columns 

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   fueltype          205 non-null    object 
 2   aspiration        205 non-null    object 
 3   doornumber        205 non-null    object 
 4   carbody           205 non-null    object 
 5   drivewheel        205 non-null    object 
 6   enginelocation    205 non-null    object 
 7   wheelbase         205 non-null    float64
 8   carlength         205 non-null    float64
 9   carwidth          205 non-null    float64
 10  carheight         205 non-null    float64
 11  curbweight        205 non-null    int64  
 12  enginetype        205 non-null    object 
 13  cylindernumber    205 non-null    object 
 14  enginesize        205 non-null    int64  
 15  fuelsystem        205 non-null    object 
 16  boreratio         205 non-null    float64
 1

In [25]:
df.describe()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [26]:
df.shape

(205, 25)

In [27]:
null_values=df.isnull().sum()
print("null values of each column:\n",null_values)

null values of each column:
 symboling           0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
CarBrand            0
dtype: int64


There is no null values in the dataset, so there is no imputation needed.

In [35]:
df.duplicated().sum()

0

There is no duplicate entry in this dataset.

In [38]:
categorical_columns=df.select_dtypes(include=["object","category"]).columns.tolist()
numerical_columns=df.select_dtypes(include=["int64","float64"]).columns.tolist()

print("categorical_columns:",categorical_columns)
print("numerical_columns:",numerical_columns)

categorical_columns: ['fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem', 'CarBrand']
numerical_columns: ['symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']


# In the dataset,there is categorical column.so opted for  Encoding categorical data using label encoder.

In [41]:
categorical_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

In [43]:
print(df)

     symboling  fueltype  aspiration  doornumber  carbody  drivewheel  \
0            3         1           0           1        0           2   
1            3         1           0           1        0           2   
2            1         1           0           1        2           2   
3            2         1           0           0        3           1   
4            2         1           0           0        3           0   
..         ...       ...         ...         ...      ...         ...   
200         -1         1           0           0        3           2   
201         -1         1           1           0        3           2   
202         -1         1           0           0        3           2   
203         -1         0           1           0        3           2   
204         -1         1           1           0        3           2   

     enginelocation  wheelbase  carlength  carwidth  ...  fuelsystem  \
0                 0       88.6      168.8      64.1

In [45]:
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target

select_k = SelectKBest(score_func=f_classif, k=10)  # Selecting top 10 features
X_selected = select_k.fit_transform(X, y)

# Get selected feature names and scores
selected_features = X.columns[select_k.get_support()]
selected_scores = select_k.scores_[select_k.get_support()]

print("Selected Features:", selected_features)
print("Feature Scores based on select_k:", selected_scores)


# Create a DataFrame to display feature names and scores
feature_scores_df = pd.DataFrame({'Feature': selected_features, 'Score': selected_scores})


# Sort by scores in ascending order
feature_scores_df = feature_scores_df.sort_values(by="Score", ascending=False)

# Print results
print("Selected Features:\n", feature_scores_df)

Selected Features: Index(['enginelocation', 'wheelbase', 'carlength', 'carwidth', 'curbweight',
       'cylindernumber', 'enginesize', 'horsepower', 'citympg', 'highwaympg'],
      dtype='object')
Feature Scores based on select_k: [        inf 10.27215716 17.04739001 18.60420579 36.89334129 21.90472237
 40.73817399 17.35176432 10.45495331  7.98691047]
Selected Features:
           Feature      Score
0  enginelocation        inf
6      enginesize  40.738174
4      curbweight  36.893341
5  cylindernumber  21.904722
3        carwidth  18.604206
7      horsepower  17.351764
2       carlength  17.047390
8         citympg  10.454953
1       wheelbase  10.272157
9      highwaympg   7.986910


C:\Users\ajaya\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


In [47]:
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target

In [49]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print("\n Training data(features):")
print(X_train)
print("\n Testing data(features):")
print(X_test)


 Training data(features):
     symboling  fueltype  aspiration  doornumber  carbody  drivewheel  \
66           0         0           0           0        3           2   
111          0         1           0           0        3           2   
153          0         1           0           0        4           1   
96           1         1           0           0        3           1   
38           0         1           0           1        2           1   
..         ...       ...         ...         ...      ...         ...   
106          1         1           0           1        2           2   
14           1         1           0           0        3           2   
92           1         1           0           0        3           1   
179          3         1           0           1        2           2   
102          0         1           0           0        4           1   

     enginelocation  wheelbase  carlength  carwidth  ...  enginesize  \
66                0     

In [51]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
X_train_scaled

array([[-0.72738032, -3.15171911, -0.46348119, ...,  0.96431609,
         1.3135007 , -0.45585288],
       [-0.72738032,  0.31728716, -0.46348119, ..., -0.98812636,
        -1.02456857,  0.08198793],
       [-0.72738032,  0.31728716, -0.46348119, ...,  0.96431609,
         1.00175813,  1.02320934],
       ...,
       [ 0.07863571,  0.31728716, -0.46348119, ...,  0.96431609,
         1.00175813, -0.05247227],
       [ 1.69066776,  0.31728716, -0.46348119, ..., -0.98812636,
        -1.02456857,  1.02320934],
       [-0.72738032,  0.31728716, -0.46348119, ..., -1.31353343,
        -1.33631113, -0.05247227]])

# 2. Model Implementation
 Implement the following five regression algorithms:
1) Linear Regression
2) Decision Tree Regressor
3) Random Forest Regressor
4) Gradient Boosting Regressor
5) Support Vector Regressor

In [56]:
models={
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(),
    "Support Vector Regressor": SVR()
}

# 3. Model Evaluation :
-- Compare the performance of all the models based on R-squared, Mean Squared Error (MSE), and Mean Absolute Error (MAE).
-- Identify the best performing model and justify why it is the best.

In [59]:
model_results={}

for name,model in models.items():
    model.fit(X_train,y_train)
    predictions=model.predict(X_test)
    # Calculate evaluation metrics
    mae=mean_absolute_error(y_test,predictions)
    mse=mean_squared_error(y_test,predictions)
    r2=r2_score(y_test,predictions)
    rmse=mean_squared_error(y_test,predictions,squared=False)
    model_results[name]={"MAE":mae,"MSE":mse,"R2":r2,"RMSE":rmse}

In [61]:
for name,metrics in model_results.items():
    print(f"\n{name} performance:")
    for metric,value in metrics.items():
        print(f"{metric}: {value:.4f}")


Linear Regression performance:
MAE: 2147.1212
MSE: 12683093.8526
R2: 0.8393
RMSE: 3561.3332

Decision Tree Regressor performance:
MAE: 1865.0041
MSE: 7661608.7464
R2: 0.9029
RMSE: 2767.9611

Random Forest Regressor performance:
MAE: 1319.8305
MSE: 3475084.8414
R2: 0.9560
RMSE: 1864.1579

Gradient Boosting Regressor performance:
MAE: 1682.6140
MSE: 5774288.5314
R2: 0.9269
RMSE: 2402.9749

Support Vector Regressor performance:
MAE: 5707.0388
MSE: 86992751.2547
R2: -0.1020
RMSE: 9326.9905


In [63]:
best_model=max(model_results,key=lambda x: model_results[x]["R2"])
worst_model=min(model_results,key=lambda x: model_results[x]["R2"])

print("best_model:", best_model,model_results[best_model])
print("worst_model:", worst_model,model_results[worst_model])

best_model: Random Forest Regressor {'MAE': 1319.8304593495934, 'MSE': 3475084.8413772555, 'R2': 0.9559803921480681, 'RMSE': 1864.1579443215792}
worst_model: Support Vector Regressor {'MAE': 5707.03883674351, 'MSE': 86992751.25465356, 'R2': -0.10195490786142458, 'RMSE': 9326.990471457208}


# 4. Feature Importance Analysis:
Identify the significant variables affecting car prices (feature selection)

In [66]:
# Feature Importance (Random Forest)
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_scaled, y_train)
feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop 10 Important Features:\n", feature_importances.head(10))


Top 10 Important Features:
 enginesize    0.554027
curbweight    0.296704
highwaympg    0.045578
horsepower    0.027877
CarBrand      0.013995
carwidth      0.013034
carlength     0.007644
wheelbase     0.006435
peakrpm       0.005469
citympg       0.005055
dtype: float64


# 5. Hyperparameter Tuning :
Perform hyperparameter tuning and check whether the performance of the model has increased.

For small dataset here using Gradientsearchcv for hyperparametertuning 

In [70]:
models2 = {
     
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "SVR": SVR()
}
params = {
    "Decision Tree": {"max_depth": [3, 5, None]},
    "Random Forest": {"n_estimators": [50, 100]},
    "Gradient Boosting": {"n_estimators": [50, 100]},
    "SVR": {"C": [0.1, 1], "kernel": ["linear"]}
}
# Evaluate models
results = []
for name, model in models2.items():
    grid = GridSearchCV(model, params[name], cv=3, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.best_estimator_.predict(X_test)
    results.append({"Model": name, "R2 Score": r2_score(y_test, y_pred)})

# Display results
results_df = pd.DataFrame(results).sort_values(by="R2 Score", ascending=False)
print(results_df)

               Model  R2 Score
1      Random Forest  0.955809
2  Gradient Boosting  0.927835
0      Decision Tree  0.886958
3                SVR  0.823667


There is a slight variations in the model performance as compared to before hyper parameter tuning.

Since `LinearRegression()` doesn't have hyperparameters for tuning, we use **Ridge Regression**, which allows tuning of the `alpha` parameter (L2 regularization). 


The End